In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import os
import pandas as pd
import re
from sklearn.metrics import confusion_matrix

In [3]:
directory1= r"/home/daddy_yankee/Desktop/csn371/csn371_assignment/Train-corpus"
directory2=r"/home/daddy_yankee/Desktop/csn371/csn371_assignment/Test-corpus"

In [4]:
word_dict=dict()
tag_dict=dict()
words_tags_dictionary = dict()
tag_list=list()
word_list=list()
all_tags=list()

In [5]:
def tag2_given_tag1(t2,t1,train_tag=tag_list,var=tag_dict):
    count_t1=var[t1]
    count_t2_and_t1=0
    for i in range(len(train_tag)-1):
        if train_tag[i]==t1 and train_tag[i+1]==t2 :
            count_t2_and_t1+=1
    return count_t2_and_t1/count_t1

In [6]:
def prob_word_given_tag(word, tag,tag_dict,words_tags_dictionary):
    string= str(word)+"_"+str(tag)
    try:
     count= words_tags_dictionary[string]
     return count/(tag_dict[tag])
    except:
     return 0

In [7]:
def viterbi(test_words, all_tags, train_bag=words_tags_dictionary,train_list_tags=tag_list ,var1=word_dict):
    state=[]
    T=list(all_tags)
    likelihood=dict()
    for key,word in enumerate(test_words):
        likelihood[key]=dict()
        p=[]

        for tag in T:
            if word in var1.keys():
                emission_prob=prob_word_given_tag(word,tag,tag_dict,words_tags_dictionary)
            else:
                emission_prob=0

            if key==0:
                transition_prob=tags_df.loc['PUN',tag]
                likelihood[key][tag]=emission_prob*transition_prob
                p.append(likelihood[key][tag])
            else:
                trans_states=[]
                for prev_tags in all_tags:
                    transition_prob=tags_df.loc[prev_tags,tag]
                    tag_likelihood=transition_prob*emission_prob*likelihood[key-1][prev_tags]
                    trans_states.append(tag_likelihood)
                max_tag_prob=max(trans_states)
                likelihood[key][tag]=max_tag_prob
                p.append(likelihood[key][tag])

        most_likely_tag_prob=max(p)
        final_tag_index=p.index(most_likely_tag_prob)
        state.append(all_tags[final_tag_index])
    return list(state)

In [8]:
for file in os.listdir(directory1):
    subfile = os.path.join(directory1, file)
    for xml_files in os.listdir(subfile):
        xml_file = os.path.abspath(os.path.join(subfile, xml_files))
        myTree = ET.parse(xml_file)
        myRoot = myTree.getroot()
        for x in myRoot.findall('.//w'):
            text=(x.text).strip().lower()
            tag=x.attrib['c5']
            tag=re.split('-',tag)[0]


            tag_list.append(tag)
            word_list.append(text)

            if tag in tag_dict:
                tag_dict[tag]+=1
            else:
                tag_dict[tag]=1

            if text in word_dict:
                word_dict[text] += 1
            else:
                word_dict[text] = 1

            string = str(text)+'_'+ str(tag)

            if string in words_tags_dictionary:
                words_tags_dictionary[string]+=1
            else:
                words_tags_dictionary[string]=1

        for x in myRoot.findall('.//c'):
            text = (x.text).strip().lower()
            tag = x.attrib['c5']
            tag = re.split('-', tag)[0]

            tag_list.append(tag)
            word_list.append(text)

            if tag in tag_dict:
                tag_dict[tag] += 1
            else:
                tag_dict[tag] = 1

            if text in word_dict:
                word_dict[text] += 1
            else:
                word_dict[text] = 1

            string = str(text) + '_' + str(tag)

            if string in words_tags_dictionary:
                words_tags_dictionary[string] += 1
            else:
                words_tags_dictionary[string] = 1

all_tags=list(tag_dict.keys())
print(all_tags)
all_words=list(word_dict.keys())

['AJ0', 'NN2', 'AVP', 'PRP', 'NP0', 'CJC', 'VVD', 'ORD', 'NN1', 'CJT', 'AT0', 'VBD', 'AV0', 'POS', 'NN0', 'PRF', 'PNP', 'VDD', 'XX0', 'VVI', 'VVG', 'TO0', 'VM0', 'AJC', 'CJS', 'VHD', 'VVN', 'DTQ', 'AVQ', 'VBZ', 'DT0', 'VVB', 'VBB', 'PNQ', 'VHZ', 'VVZ', 'AJS', 'DPS', 'VBI', 'VHB', 'VBN', 'CRD', 'UNC', 'PNI', 'EX0', 'VHI', 'VDB', 'VBG', 'PNX', 'VDI', 'VHN', 'VDN', 'VDZ', 'ITJ', 'PUQ', 'PUN', 'PUL', 'PUR', 'VHG', 'ZZ0', 'VDG']


In [9]:
transition_probabilities=np.zeros((len(all_tags),len(all_tags)),dtype='float32')
for i,tag1 in enumerate(all_tags):
    for j,tag2 in enumerate(all_tags):
        transition_probabilities[i,j]=tag2_given_tag1(tag2,tag1)

tags_df=pd.DataFrame(transition_probabilities, columns=all_tags, index= all_tags)
print(tags_df)


          AJ0       NN2       AVP       PRP       NP0       CJC       VVD  \
AJ0  0.074898  0.191886  0.000451  0.054957  0.022923  0.047696  0.002150   
NN2  0.020599  0.020183  0.004559  0.180793  0.024326  0.108806  0.030861   
AVP  0.020647  0.016676  0.004655  0.352335  0.021439  0.067148  0.005112   
PRP  0.071782  0.041302  0.000284  0.003090  0.118005  0.001877  0.000176   
NP0  0.020683  0.022701  0.001266  0.064668  0.288174  0.065525  0.067144   
CJC  0.103480  0.061048  0.003466  0.049075  0.073754  0.000242  0.047514   
VVD  0.038606  0.018241  0.066963  0.152247  0.053188  0.018775  0.002208   
ORD  0.053768  0.030762  0.001405  0.039324  0.051036  0.023943  0.016708   
NN1  0.016066  0.047245  0.003823  0.162902  0.034984  0.083056  0.024401   
CJT  0.030587  0.037302  0.000518  0.036503  0.068953  0.000621  0.036976   
AT0  0.259235  0.077754  0.000057  0.000174  0.043168  0.000031  0.000014   
VBD  0.128546  0.009018  0.009482  0.050376  0.013884  0.002994  0.000250   

In [10]:
words_tags_mat = np.zeros((len(all_tags), len(all_words)),dtype='float32')
for i, tag in enumerate(all_tags):
    for j, word in enumerate(all_words):
        words_tags_mat[i, j] = prob_word_given_tag(word, tag,tag_dict,words_tags_dictionary)

condition_probabilities = pd.DataFrame(words_tags_mat, columns=all_words, index=all_tags)
print(condition_probabilities)


      angry    tories     round        on    lawson        by   anthony  \
AJ0  0.0005  0.000000  0.000344  0.000000  0.000000  0.000000  0.000000   
NN2  0.0000  0.001162  0.000000  0.000000  0.000000  0.000000  0.000000   
AVP  0.0000  0.000000  0.013671  0.089070  0.000000  0.005621  0.000000   
PRP  0.0000  0.000000  0.001207  0.073633  0.000000  0.069397  0.000000   
NP0  0.0000  0.000000  0.000004  0.000000  0.000733  0.000000  0.000843   
CJC  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VVD  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
ORD  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NN1  0.0000  0.000000  0.000319  0.000000  0.000000  0.000001  0.000000   
CJT  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
AT0  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VBD  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
AV0  0.0000  0.000000  0.

In [ ]:
test_tags=dict()
test_words=dict()
test_tags_list=list()
test_words_list=list()
final_test_results={}
for file in os.listdir(directory2):
    subfile = os.path.join(directory2, file)
    for xml_files in os.listdir(subfile):
        xml_file = os.path.abspath(os.path.join(subfile, xml_files))
        myTree = ET.parse(xml_file)
        myRoot = myTree.getroot()
        for x in myRoot.findall('.//w'):
            text=(x.text).strip().lower()
            w= re.split('-',x.attrib['c5'])[0]

            if text in test_words:
                test_words[text]+=1
            else:
                test_words[text]=1

            if w in test_tags:
                test_tags[w]+=1
            else:
                test_tags[w]=1

            test_tags_list.append(w)
            test_words_list.append(text)
        for x in myRoot.findall('.//c'):
            text = (x.text).strip().lower()
            w = re.split('-', x.attrib['c5'])[0]

            if text in test_words:
                test_words[text] += 1
            else:
                test_words[text] = 1

            if w in test_tags:
                test_tags[w] += 1
            else:
                test_tags[w] = 1

            test_tags_list.append(w)
            test_words_list.append(text)
        predicted_tags=viterbi(test_words_list, all_tags)
        final_test_results=list(predicted_tags)

final_train_results=test_tags_list
numerator=0
denominator=len(final_train_results)
for i in range(len(final_train_results)):
    if final_train_results[i]==final_test_results[i]:
        numerator+=1

print(numerator/denominator)


In [ ]:
print("Confusion Matrix = ")
print("\n")
print(confusion_matrix(final_train_results,final_test_results))
print("\n")